# Importing needed libraries

**if there are errors importing these libraries, just type in terminal : <code> pip install library_name </code>** <br> 
Sample: pip install pandas <br>
libraries used: pandas, nltk, sklearn, scipy, numpy <br>
To Do: push a requirements.txt for easy install

In [1]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial import distance
import numpy as np
import timeit
import seaborn as sns

In [2]:
# configure pandas for the datasets
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

In [3]:
# prepare stop words 
stop_words = stopwords.words('english')

# Getting Data Sources

In [4]:
raw_cv_data = pd.read_csv('UpdatedResumeDataSet.csv')
raw_job_data = pd.read_csv('dice_com-job_us_sample.csv')

# Cleaning and Preprocessing Data Sources 

**removing resume & job duplicates**

In [5]:
resume_no_dup = raw_cv_data.drop_duplicates('Resume',keep='first')

#preprocess and dropping job duplicates
jobs_data_from_raw =  raw_job_data[["jobdescription", "skills", "uniq_id", "jobtitle"]].copy()
no_duplicates_job = jobs_data_from_raw.drop_duplicates('jobdescription',keep='first')
no_duplicates_job = no_duplicates_job.drop_duplicates('skills',keep=False)

* remove the job data that contains irrelevant skill content 

In [6]:
# remove see job description data
no_duplicates_job = no_duplicates_job[no_duplicates_job["skills"].str.contains("job description", case=False)==False]
no_duplicates_job = no_duplicates_job[no_duplicates_job["skills"].str.contains("full time", case=False)==False]
no_duplicates_job = no_duplicates_job[no_duplicates_job["skills"].str.contains("see below", case=False)==False]
no_duplicates_job = no_duplicates_job[no_duplicates_job["skills"].str.contains("null", case=False)==False]

 * after manual checking, there are unneeded job data that should be removed <br> reason: I forgot 

In [7]:
no_duplicates_job = no_duplicates_job.drop([no_duplicates_job.index[11670],no_duplicates_job.index[16427],no_duplicates_job.index[18] ])

* clean data

In [8]:
# clean/preprocess and lowercase the skills column
no_duplicates_job['skills'] = no_duplicates_job['skills'].str.lower()
no_duplicates_job["skills"] = no_duplicates_job["skills"].str.replace(","," ")
no_duplicates_job["skills"] = no_duplicates_job["skills"].str.replace('"', " ")
no_duplicates_job["skills"] = no_duplicates_job["skills"].str.replace(r"[^a-zA-Z0-9+#]"," ", regex=True)

#clean jobs
no_duplicates_job['jobdescription'] = no_duplicates_job['jobdescription'].str.lower()
no_duplicates_job["jobdescription"] = no_duplicates_job["jobdescription"].str.replace(r"[^a-zA-Z0-9+#]"," ", regex=True)

# change job index
no_duplicates_job.index = no_duplicates_job["uniq_id"]

# clean resumes
resume_no_dup_cleaned = resume_no_dup.applymap(lambda s:s.lower() if type(s) == str else s)
resume_no_dup_cleaned = resume_no_dup_cleaned.replace(r"[^a-zA-Z0-9+#]",' ',regex=True)

*For double checking if duplicates were dropped* <br>
raw_cv_data.info() : 900+ <br>
resume_no_dup.info(): 166 resume <br> <br>
raw_job_data: 22000 <br>
no_duplicates_job: 18339

# Extracting Features (One hot encoding)

We need to convert the datasets into vector representations through one hot encoding <br> 
* Before that, we need the set of job skills to act as a column in our matrix <br> <br>

| Resume | Skill 1 | Skill 2 | Skill 3 | ... | Skill n |
| --- | --- | --- | --- | --- | --- |
| Resume id 1 | 1 | 0 | 1 | ... | 1 |
| Resume id 2 | 0 | 0 | 1 | ... | 1 |

<br> 

| Job | Skill 1 | Skill 2 | Skill 3 | ... | Skill n |
| --- | --- | --- | --- | --- | --- |
| Job id 1 | 1 | 0 | 1 | ... | 1 |
| Job id 2 | 0 | 0 | 1 | ... | 1 |

In [9]:
# get skills from job dataset through the skills column 
raw_skills = []

for i in no_duplicates_job['skills']:
    raw_skills.extend(i.split(' '))
    
# get the set of skills 
skills_set = set(raw_skills)
skills_set.remove('')

# remove stop words
for i in stop_words:
    skills_set.discard(i)

In [10]:
# let's validate that these set of skills are the same as what we have used in the TF-IDF program

skill_file = open('a_file.txt', 'r')
skills_from_file = []

for i in skill_file:
    skills_from_file.append(str.rstrip(i))
error = 0
for i in skills_set:
    if i not in skills_from_file:
        print('Error: Can not find this word: {word}'.format(word=i))
print('No errors all good')

No errors all good


In [11]:
# let's just use the skills from the TF-IDF program because I don't know if the order matters, let's just play safe
skills_set = skills_from_file.copy()

**Let's extract features** <br>
* prepare a dataframe for resume and jobs

In [12]:
resume_vectors = pd.DataFrame(index=resume_no_dup_cleaned.index, columns=skills_set)
job_vectors = pd.DataFrame(index=no_duplicates_job.index, columns=skills_set)

* vectorize resume

In [13]:
for i in resume_vectors.index:
    current_resume_content = resume_no_dup_cleaned.loc[i]['Resume']
    resume_vectors.loc[i] = [1 if skill in current_resume_content else 0 for skill in skills_set]

In [14]:
resume_vectors

,bam,jqueryjava,gravity,prediscovery,quickbooks,veteran,skils,petabytes,actionscript,variable,schneider,pp,implementations,middle,trizetto,play,password,raleigh,mgt,iiq,symbol,domumentum,kmv,custom,bond,dynamics,itil,19103,flask,redseal,unable,idd,ccsk,es2015,novell,gold,wire,jabber,conf,tao,...,hcissp,capture,crx,abg,hockey,activities,quote,tranfer,ncpdp,documents,parametric,paralegal,dashboarding,solidworks,electrical,lot,v10,vin,military,gaming,fireeye,microelectronics,electron,13c,bonus,safetrace,com,consultant,maintaining,vrealise,udm,delta,asme,4+years,exper,z,acceptable,thorough,seven,physical
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
895,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
896,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
897,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


* vectorize jobs

In [15]:
for i in job_vectors.index:
    current_jobdescription = no_duplicates_job.loc[i]['jobdescription']
    job_vectors.loc[i] = [1 if skill in current_jobdescription else 0 for skill in skills_set]

In [16]:
job_vectors

,bam,jqueryjava,gravity,prediscovery,quickbooks,veteran,skils,petabytes,actionscript,variable,schneider,pp,implementations,middle,trizetto,play,password,raleigh,mgt,iiq,symbol,domumentum,kmv,custom,bond,dynamics,itil,19103,flask,redseal,unable,idd,ccsk,es2015,novell,gold,wire,jabber,conf,tao,...,hcissp,capture,crx,abg,hockey,activities,quote,tranfer,ncpdp,documents,parametric,paralegal,dashboarding,solidworks,electrical,lot,v10,vin,military,gaming,fireeye,microelectronics,electron,13c,bonus,safetrace,com,consultant,maintaining,vrealise,udm,delta,asme,4+years,exper,z,acceptable,thorough,seven,physical
uniq_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8aec88cba08d53da65ab99cf20f6f9d9,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
46baa1f69ac07779274bcd90b85d9a72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0
45efa1f6bc65acc32bbbb953a1ed13b7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
e0ac9d926dda5e95162ef05adea7318c,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0
e7e326053c586bd94e59f1fd74de4a1b,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1acc1a7a845f0b9c233ad42941f0462d,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
86e27ce6b7e631e55d69d142c7d43df2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0
4287c7ee3317ccf1edd76e238cf8e584,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0


**RESUME AND JOB VECTORS FOR NN**

In [17]:
#resume_vectors_NN = pd.DataFrame(index=range(0, 166), columns=skills_set).fillna(0)

In [18]:
#resume_vectors_NN

# Generating List of Recommendations for all resumes

**WE-UCF (cosine)**

In [19]:
# user-item matrix (WE-UCF cosine)
resume_job_cos_sim = cosine_similarity(resume_vectors, job_vectors)
resume_job_matrix = pd.DataFrame(resume_job_cos_sim, index=resume_vectors.index ,columns = job_vectors.index)

In [20]:
resume_job_matrix

uniq_id,8aec88cba08d53da65ab99cf20f6f9d9,46baa1f69ac07779274bcd90b85d9a72,45efa1f6bc65acc32bbbb953a1ed13b7,e0ac9d926dda5e95162ef05adea7318c,e7e326053c586bd94e59f1fd74de4a1b,b0dadecf4c3c2beecb9c773ca11ecda4,28f5e0c1cc3314813e674f0c32b04d1b,5e0ff38f5eaf44726f4e3d1dd257a244,e4a1ff1b6c0fda5f345e57cf1acb40dd,d0c81a2e3e5d666f3d730f1048c49132,51279d060da242e3baea98b26ddd641e,48b7ca0c2f6191fb48e7ecf19fbd3322,7ad2eadde69e07fee0e38c1a251dd81f,f3af6886ca0d133abda2ddf9b84633e4,d3073d47b79938269b22bdea4dc0b9b8,b7fab2d3de5e129310b382d8f51508f6,4868383e3f99535778f354fcb734d57c,81f8732626888727fcc1093b6c084839,1bef3cf6259e80d18222407858ad9052,72a3f0f0f24a0a78c5fac3c2376f5979,4a3f8742fae151757eb290a774968371,b1db84e23f424b1481b29212749abadc,51a0050d469e787dba2102383797536d,f2ade5d053b992593e2bdacd83a18a7f,2831bbdba025f87efca38b9761f26b95,2b2322459de27834b0c3c86737fb16a8,b923d29624fb4dc9e193aa30f8253ee6,5be5533c9ed448d1ef92016a56573869,ebdaa795cb443c6b7881b7a15c4c25b1,d92f267244249065890ee6bde5275f49,8cbcc9ef3be25e4409fcb4ab55d6503d,e2f1a046c937631529aec11a56f8590a,a2ef697606d460795dda16de672a5d5d,d1130a38c05cd6f6df40a2636a76ea5b,8cb4914f42c5b805d19be0952b3c88e9,8c50099ba0e24e089c12c6e68bb2bdc6,d6920e2979999da6d892774e2be97b75,065002d37b05ad5fa6478b709b94a3a4,b2db6c4923a59708ff6c562b08c23c73,d997f70cf7082ec18ffbb3e7c382001c,...,263e10b8aa545c6406211bb06136e787,a2f612a294080ff97b26ea18ee9ef62c,b24a51456f6575effeec94fb8c34f318,14945b616ed0d0be7a90cd0aa11d5614,e2c8714be39ad48a4f899fc327d99325,993e98d89cc6657df46f6bf6c84f2ca7,3184538e7963c8d5b5f87c34a84fb023,2f2e09bd5cab7c513cf982d68426ad2f,7b94bee8c0b73184ec83e0325900e5e6,17a9c1590e0f878e7ac25e0156287dfc,4574705b4e9054c48f01aacc0efa3964,dc9018c1e30468233e0220e4ab7fa22f,0cce919bcf70c6015ff1c71bea138438,03d79c5f3413b424fefb0a69fbba6ea0,5a47b4ab7731f842c60e5405455b9842,eb93c8d237a8aacc2b741517f40ee190,e501c02a4038dd49d420411fc2632ac8,3a0534473ab9c5cc03899a77703bb9e2,83d1393d14ae7edbefb719b5f37ecd6d,f030dd810d86411a93cb5682c0ac35aa,fd1e05e481af4a76fcc5a7c49914825b,788d893cd6b11dfd5ea921666ee1e009,b6b3d382ba66dfe7f80abaaa89a92e3d,ab42980a1be610f35939d37527fb72c2,8c4e3cfa8afc91c1d002e881b6493b29,c5fc49888e42880339d7753fd7559990,fc7085f0ab79099a6d87490d27def0c8,6cae8f5b52c79e884168d6f9b4962025,74262c496c6c04e38a7fbbb4e2e500d4,95bebff86da8f10232e75d24a94297d3,9a415ba24303aa16475f85438fc78921,76f85e4cf8b60279811613c7cbe72663,ba32786b50e8b4d468cb9d813066fea5,541b0f16ecd86fdbd7ee2b04523dc65e,ee46660dd5318edb27b229f51341fcf8,1acc1a7a845f0b9c233ad42941f0462d,86e27ce6b7e631e55d69d142c7d43df2,4287c7ee3317ccf1edd76e238cf8e584,d7512f0181d69f83f96db38cd77a4d08,ec375268b494b3bcbed1635d64226112
0,0.483155,0.515483,0.422187,0.337680,0.457980,0.482045,0.484705,0.518270,0.542398,0.457705,0.480854,0.438580,0.503884,0.469288,0.514115,0.511118,0.442318,0.495947,0.487173,0.387824,0.500502,0.504497,0.468418,0.467683,0.478625,0.508189,0.379517,0.517972,0.472568,0.460806,0.472949,0.437806,0.495810,0.443383,0.477730,0.505560,0.407868,0.436610,0.446973,0.519465,...,0.447867,0.488958,0.310930,0.506509,0.463354,0.451813,0.363039,0.470306,0.368123,0.446697,0.494966,0.485994,0.382931,0.416921,0.419649,0.495818,0.462872,0.385486,0.412484,0.526199,0.460537,0.449248,0.457854,0.469153,0.455743,0.441778,0.302848,0.419607,0.495800,0.411116,0.245944,0.458117,0.354734,0.381339,0.452127,0.403994,0.519176,0.473767,0.520379,0.501252
1,0.443722,0.411592,0.409967,0.365997,0.393545,0.396598,0.441472,0.437451,0.420469,0.434214,0.428290,0.388896,0.388674,0.423091,0.437462,0.397334,0.423411,0.439835,0.409248,0.400820,0.417866,0.399382,0.389952,0.393103,0.446057,0.448841,0.373557,0.415744,0.409894,0.446658,0.397191,0.429102,0.391693,0.439474,0.417251,0.415424,0.482252,0.406212,0.434221,0.415855,...,0.397732,0.445003,0.334207,0.408412,0.438233,0.426908,0.384206,0.425822,0.369168,0.419305,0.409615,0.454984,0.380030,0.422780,0.435887,0.422256,0.406212,0.393272,0.416354,0.422496,0.412648,0.424646,0.430126,0.375201,0.414937,0.430517,0.397229,0.470923,0.42

* Figures


In [21]:
#ax = sns.heatmap(resume_job_cos_sim, cmap = 'coolwarm', linecolor = 'white', xticklabels=job_vectors.index, yticklabels=resume_vectors.index)
#ax.tick_params(length=0)
#ax.xaxis.tick_top()
#figure = ax.get_figure()
#figure.savefig('Cosine similarity of each applicant’s resume and employer’s job description.png',dpi=400)

In [22]:
# user - user matrix (WE-UCF cosine)
resume_resume_cos_sim = cosine_similarity(resume_job_matrix, resume_job_matrix)
resume_resume_matrix = pd.DataFrame(resume_resume_cos_sim, index=resume_job_matrix.index, columns=resume_job_matrix.index)

In [23]:
resume_resume_matrix

,0,1,2,3,4,5,6,7,8,9,40,41,42,43,45,46,47,48,49,50,84,85,86,87,88,89,90,91,92,93,104,105,106,107,108,109,140,141,143,144,...,711,712,713,714,715,716,717,718,719,742,743,744,745,746,747,748,784,785,786,787,788,824,825,826,827,828,829,830,852,853,854,855,856,892,893,894,895,896,897,898
0,1.000000,0.994460,0.996487,0.998962,0.985815,0.990603,0.996244,0.998931,0.995690,0.998887,0.986509,0.997653,0.979593,0.981161,0.979112,0.981493,0.993256,0.990082,0.993955,0.994206,0.995427,0.985899,0.977976,0.988381,0.997764,0.992349,0.988577,0.993334,0.988999,0.983562,0.995717,0.989154,0.998062,0.985963,0.989925,0.992807,0.997778,0.997855,0.989777,0.998732,...,0.997941,0.998124,0.995338,0.998031,0.998127,0.997755,0.998230,0.998116,0.997603,0.998740,0.997105,0.998514,0.998420,0.999120,0.995211,0.998996,0.997980,0.998820,0.997601,0.998818,0.991942,0.991683,0.987166,0.997197,0.985100,0.998067,0.999046,0.995454,0.996394,0.998368,0.994759,0.996940,0.997159,0.993007,0.995540,0.995761,0.997759,0.997425,0.991848,0.995798
1,0.994460,1.000000,0.998089,0.993814,0.995818,0.997950,0.998341,0.995587,0.998495,0.995794,0.996771,0.997060,0.993107,0.993595,0.992694,0.993213,0.998224,0.997787,0.997841,0.997696,0.998401,0.996447,0.992494,0.997458,0.995839,0.998332,0.997179,0.998562,0.997489,0.995344,0.998137,0.997505,0.991720,0.996362,0.998016,0.998598,0.996919,0.996574,0.997562,0.995859,...,0.994709,0.995535,0.997586,0.995863,0.994771,0.995622,0.994807,0.995194,0.996654,0.995883,0.997853,0.996511,0.996061,0.995208,0.997171,0.995466,0.996487,0.994743,0.997611,0.994423,0.998193,0.996833,0.995692,0.997745,0.995422,0.996111,0.994890,0.997492,0.998190,0.996458,0.997891,0.996773,0.997805,0.997613,0.998161,0.998197,0.996389,0.996708,0.997448,0.997917
2,0.996487,0.998089,1.000000,0.995841,0.993702,0.996699,0.998205,0.997704,0.998623,0.996976,0.994640,0.997608,0.989469,0.990240,0.988852,0.990624,0.997617,0.995561,0.997885,0.998417,0.997917,0.993469,0.988420,0.995343,0.996775,0.996773,0.995445,0.997085,0.995277,0.992376,0.998481,0.995748,0.993805,0.993955,0.995665,0.997515,0.998417,0.998474,0.996125,0.997850,...,0.996341,0.996830,0.997485,0.997374,0.996396,0.996927,0.996358,0.996640,0.997494,0.997338,0.998556,0.997870,0.998004,0.997016,0.998371,0.997209,0.997517,0.996279,0.998196,0.996298,0.997273,0.996877,0.994485,0.998464,0.993941,0.997952,0.997152,0.998275,0.998220,0.998113,0.998705,0.998639,0.998869,0.997143,0.998768,0.998033,0.997275,0.996866,0.997503,0.998394
3,0.998962,0.993814,0.995841,1.000000,0.984571,0.989686,0.995983,0.998719,0.995104,0.998890,0.985638,0.997863,0.978457,0.980061,0.978095,0.980223,0.992723,0.989600,0.993395,0.993555,0.995327,0.984966,0.976790,0.987584,0.998033,0.991773,0.987569,0.992945,0.988472,0.982566,0.995674,0.988720,0.998900,0.985041,0.989264,0.992437,0.997186,0.997061,0.988858,0.998606,...,0.998038,0.998329,0.995216,0.998012,0.998284,0.997937,0.998339,0.998294,0.997598,0.998640,0.996752,0.998324,0.998057,0.999196,0.994155,0.998890,0.998188,0.999206,0.997665,0.999040,0.991367,0.990245,0.985470,0.996657,0.984150,0.997588,0.998838,0.994343,0.996142,0.997976,0.993752,0.996246,0.996417,0.992887,0.994843,0.995687,0.998043,0.997812,0.991183,0.995455
4,0.985815,0.995818,0.993702,0.984571,1.000000,0.997228,0.993902,0.987690,0.994607,0.987632,0.998050,0.990449,0.997315,0.995979,0.996926,0.996316,0.996662,0.996207,0.994811,0.995700,0.993996,0.997890,0.997216,0.997511,0.988384,0.996247,0.997415,0.995736,0.997247,0.996978,0.994026,0.996940,0.981142,0.997272,0.996470,0.995945,0.991018,0.990679,0.996963,0.988609,...,0.987454,0.988410,0.994059,0.989172,0.987215,0.988927,0.986771,0.987856,0.990580,0.988670,0.992959,0.989422,0.989064,0.986936,0.993750,0.987471,0.990013,0.986193,0.991692,0.986218,0.996572,0.995495,0.996675,0.992721,0.996850,0.989724,0.986774,0.994211,0.993322,0.989874,0.995207,0.991633,0.992503,0.995101,0.994365,0.993866,0.989578,0.989611,0.995972,0.994169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

**WE-UCF (jaccard)**

In [24]:
# user-item matrix (WE-UCF jaccard)
resume_job_jacc = 1 - pairwise_distances(resume_vectors.values, job_vectors.values, metric="jaccard", n_jobs=-1)
resume_job_jacc_matrix = pd.DataFrame(resume_job_jacc, index=resume_vectors.index, columns=job_vectors.index)

/home/lapi/thesis/Untitled Folder/onehot/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [25]:
resume_job_jacc_matrix

uniq_id,8aec88cba08d53da65ab99cf20f6f9d9,46baa1f69ac07779274bcd90b85d9a72,45efa1f6bc65acc32bbbb953a1ed13b7,e0ac9d926dda5e95162ef05adea7318c,e7e326053c586bd94e59f1fd74de4a1b,b0dadecf4c3c2beecb9c773ca11ecda4,28f5e0c1cc3314813e674f0c32b04d1b,5e0ff38f5eaf44726f4e3d1dd257a244,e4a1ff1b6c0fda5f345e57cf1acb40dd,d0c81a2e3e5d666f3d730f1048c49132,51279d060da242e3baea98b26ddd641e,48b7ca0c2f6191fb48e7ecf19fbd3322,7ad2eadde69e07fee0e38c1a251dd81f,f3af6886ca0d133abda2ddf9b84633e4,d3073d47b79938269b22bdea4dc0b9b8,b7fab2d3de5e129310b382d8f51508f6,4868383e3f99535778f354fcb734d57c,81f8732626888727fcc1093b6c084839,1bef3cf6259e80d18222407858ad9052,72a3f0f0f24a0a78c5fac3c2376f5979,4a3f8742fae151757eb290a774968371,b1db84e23f424b1481b29212749abadc,51a0050d469e787dba2102383797536d,f2ade5d053b992593e2bdacd83a18a7f,2831bbdba025f87efca38b9761f26b95,2b2322459de27834b0c3c86737fb16a8,b923d29624fb4dc9e193aa30f8253ee6,5be5533c9ed448d1ef92016a56573869,ebdaa795cb443c6b7881b7a15c4c25b1,d92f267244249065890ee6bde5275f49,8cbcc9ef3be25e4409fcb4ab55d6503d,e2f1a046c937631529aec11a56f8590a,a2ef697606d460795dda16de672a5d5d,d1130a38c05cd6f6df40a2636a76ea5b,8cb4914f42c5b805d19be0952b3c88e9,8c50099ba0e24e089c12c6e68bb2bdc6,d6920e2979999da6d892774e2be97b75,065002d37b05ad5fa6478b709b94a3a4,b2db6c4923a59708ff6c562b08c23c73,d997f70cf7082ec18ffbb3e7c382001c,...,263e10b8aa545c6406211bb06136e787,a2f612a294080ff97b26ea18ee9ef62c,b24a51456f6575effeec94fb8c34f318,14945b616ed0d0be7a90cd0aa11d5614,e2c8714be39ad48a4f899fc327d99325,993e98d89cc6657df46f6bf6c84f2ca7,3184538e7963c8d5b5f87c34a84fb023,2f2e09bd5cab7c513cf982d68426ad2f,7b94bee8c0b73184ec83e0325900e5e6,17a9c1590e0f878e7ac25e0156287dfc,4574705b4e9054c48f01aacc0efa3964,dc9018c1e30468233e0220e4ab7fa22f,0cce919bcf70c6015ff1c71bea138438,03d79c5f3413b424fefb0a69fbba6ea0,5a47b4ab7731f842c60e5405455b9842,eb93c8d237a8aacc2b741517f40ee190,e501c02a4038dd49d420411fc2632ac8,3a0534473ab9c5cc03899a77703bb9e2,83d1393d14ae7edbefb719b5f37ecd6d,f030dd810d86411a93cb5682c0ac35aa,fd1e05e481af4a76fcc5a7c49914825b,788d893cd6b11dfd5ea921666ee1e009,b6b3d382ba66dfe7f80abaaa89a92e3d,ab42980a1be610f35939d37527fb72c2,8c4e3cfa8afc91c1d002e881b6493b29,c5fc49888e42880339d7753fd7559990,fc7085f0ab79099a6d87490d27def0c8,6cae8f5b52c79e884168d6f9b4962025,74262c496c6c04e38a7fbbb4e2e500d4,95bebff86da8f10232e75d24a94297d3,9a415ba24303aa16475f85438fc78921,76f85e4cf8b60279811613c7cbe72663,ba32786b50e8b4d468cb9d813066fea5,541b0f16ecd86fdbd7ee2b04523dc65e,ee46660dd5318edb27b229f51341fcf8,1acc1a7a845f0b9c233ad42941f0462d,86e27ce6b7e631e55d69d142c7d43df2,4287c7ee3317ccf1edd76e238cf8e584,d7512f0181d69f83f96db38cd77a4d08,ec375268b494b3bcbed1635d64226112
0,0.308880,0.342910,0.237487,0.154628,0.286403,0.310377,0.298337,0.345404,0.371378,0.287081,0.314210,0.249194,0.335377,0.305743,0.345731,0.341593,0.273684,0.327402,0.317808,0.221328,0.328358,0.335683,0.297630,0.304089,0.305101,0.332370,0.200859,0.344828,0.308348,0.289017,0.293347,0.274955,0.327753,0.258166,0.304515,0.336842,0.223416,0.263889,0.276163,0.350863,...,0.268641,0.321207,0.134091,0.338710,0.294283,0.258658,0.171946,0.302026,0.194030,0.267620,0.314629,0.288817,0.192650,0.243927,0.256795,0.321531,0.284274,0.203463,0.228051,0.356856,0.291235,0.286472,0.292902,0.304762,0.285578,0.269231,0.138122,0.231931,0.316783,0.235839,0.086147,0.294170,0.171492,0.184807,0.274924,0.221390,0.350524,0.297935,0.346910,0.333042
1,0.276565,0.245186,0.257827,0.214286,0.239130,0.238095,0.281198,0.264668,0.243398,0.269795,0.254658,0.241379,0.223392,0.246495,0.253378,0.230130,0.262222,0.263092,0.243871,0.250000,0.251656,0.231591,0.234397,0.226107,0.278426,0.278642,0.228846,0.248705,0.237705,0.280179,0.244582,0.261580,0.226730,0.281362,0.255714,0.242568,0.317365,0.251969,0.271624,0.234110,...,0.246732,0.267252,0.187643,0.233708,0.270423,0.271375,0.229581,0.258412,0.225621,0.263682,0.252632,0.294333,0.230769,0.266892,0.271386,0.258020,0.251969,0.243615,0.262760,0.235234,0.251404,0.254499,0.259843,0.215130,0.254703,0.270440,0.238202,0.307692,0.26

In [26]:
# user - user matrix (WE-UCF jaccard)
resume_resume_jacc = 1 - pairwise_distances(resume_job_jacc_matrix.values, metric="jaccard", n_jobs=-1)
resume_resume_matrix_jacc = pd.DataFrame(resume_resume_jacc, index=resume_job_jacc_matrix.index, columns=resume_job_jacc_matrix.index)

/home/lapi/thesis/Untitled Folder/onehot/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [27]:
resume_resume_matrix_jacc #hmmm apay one amin

,0,1,2,3,4,5,6,7,8,9,40,41,42,43,45,46,47,48,49,50,84,85,86,87,88,89,90,91,92,93,104,105,106,107,108,109,140,141,143,144,...,711,712,713,714,715,716,717,718,719,742,743,744,745,746,747,748,784,785,786,787,788,824,825,826,827,828,829,830,852,853,854,855,856,892,893,894,895,896,897,898
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
895,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
896,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
897,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [28]:
def generate_recommendations_for(resume, matching_func):
    match_func = {
        'cosine': [resume_job_matrix, resume_resume_matrix],
        'jaccard': [resume_job_jacc_matrix, resume_resume_matrix_jacc]
    }
    
    nearest_neighbor = match_func[matching_func][1].loc[resume].sort_values(ascending=False).drop(index=resume).head(1).index[0]
    concat_target_and_neighbor = pd.concat([match_func[matching_func][0].loc[resume],match_func[matching_func][0].loc[nearest_neighbor]])
    recommendations = concat_target_and_neighbor.sort_values(ascending=False).index.drop_duplicates(keep='first') # sort then keep the first item of duplicated jobs
    return recommendations

In [29]:
def nn_generate_recommendations_for(resume):
    
    nearest_neighbor = resume_resume_matrix.loc[resume].sort_values(ascending=False).drop(index=resume).head(1).index[0]
    #concat_target_and_neighbor = pd.concat([resume_job_matrix.loc[resume],resume_job_matrix.loc[nearest_neighbor]])
    #collected_top_n_from_neighbor = resume_job_matrix.loc[nearest_neighbor].sort_values(ascending=False)
    concat_target_and_neighbor = pd.concat([resume_job_matrix.loc[resume],resume_job_matrix.loc[nearest_neighbor]])
    recommendations = concat_target_and_neighbor.groupby('uniq_id').sum().sort_values(ascending=False)
    return concat_target_and_neighbor
    

In [30]:
nn_generate_recommendations_for(894)

,0,00069cee44688d1bb1940fb0316ceba0,00073fc9ef7e7cd3a09894fc287103ed,0009c034a443fccd0758c7add365615c,0010716804e013f363f291ace1acbff0,0012bb53511dc9144cf78e4b1ad6ca4e,001a21d7e4ccd088dabb994469039cc9,001f806d32dd936e82ebf3f3c428215d,0020d14f28f84e5bfb81a1b55527cc04,00221f996cc8b17340157f32c37329fa,00256e1f6a82d9dc26a24b44f92b0812,00265cbedbe72acdc68e9eaeb6f2f4c3,002bb85517c53686da6d5a908ffd14fe,002ca6276cf0a779a7235f81158e4940,003278fcd3a39bef1f4b3ed27c17bac6,003d9322e122a5062b7d85add07f6768,003ff19533f70734bc7667e33060246d,0049c50f720c6a8feb967b4d7f9e837b,004bd080bf26782e1297ddb100e1aff4,005415ce57a49d1cc8732435431b4907,0054cfe57ac53952453ed451632b2bff,0058e24157b64f3290a2b0131ef874e8,005a14aecf84c2dfed79e6a182be37d9,005b23cb133310ea023aa8b0decee116,0063cc1655022ba94fac6e39c32796a7,00656cf3629a582d66fa714a761e59ca,0067cc21740ced44e43af3519464e0bc,00683ac175d97b71da614eca767c3d75,007518cd08737cae0f92b796b0ad7c83,0075c441f3ec52e0227becfb750f8924,007d4edff270f66c725e1808e5a067a5,008348275ffbd16607a15bf55bae54ac,0086d52417397029e558dee2373ffeea,0089ab7dde424ea7008144244f877eed,008e0d7d6b47fba718597c52f7a6e24a,0092fc9702805959328d86e028f0ef7f,0093dd398be4bc86da069e335b4dfed7,0098dad5e33307018cdff9485f625570,009b9b809bddd2a2ba31a5086f46e6a7,00a7522c892514ef5e323446a603c310,...,ff81c6c009c0dd630968c7005d51a9ec,ff88ec6aaa59c26d8c4017de05c05a08,ff8a71abf0691d163a5108889b609dca,ff8fc80e157ba81d8bc30c3793d13e85,ff92a73a6a85e5caeac35f00f97ff787,ff937b384cf9926189b13d9bce117712,ff9693a025ec5bd222c6687722c9285c,ff989699cf1c7cafc39f81d39241ef6b,ff9992f878f62d47d6386f1e08bf407f,ff9afb6d4f30c605d31489c494c3abbd,ff9b66aee8b6c5beb3117e34a4f7d07b,ff9b9ebbf12506a34b08eaf81d88683b,ff9e1ae14ce8a12c2fc9575cb3396320,ff9f5fa7ce35e6f4a959b9cf2bf31cb5,ffa23ac33ec8c41d5736d7f424959c4d,ffa2ff8d1b97d51295050a6e388d21c3,ffa9db8825c85c5118ab75b28c2262c4,ffaa74f20f27bd48dbb4922037f43984,ffaab94d7d17eb7ad9eb8838ddde49af,ffaedc4bd4ebb8be14f586c6ac822487,ffbb37beb05dd7cd49e42f943de6485f,ffbded17a4c21bad25947a7e6e8d4219,ffc0aa45e55eeb5d42e20e7c371dec7c,ffc644bc4e0430afd22de60bfb720bc7,ffcd2b1b9afbe73b8a41337fe044b914,ffd387ee590911ae866d0aa3f33bba55,ffd6e6eea0c46653b674ea13aa5c27da,ffdb76a48d66ee3517852c87ebc61cbf,ffde524d1ff1ddcf6d1c9ec34f0cd5ce,ffe15974d709ed7a6ce56cb69d43bc4b,ffe35c50b831191466fac84fd154bb6f,ffe589de20a385dbc264c03f4d69f9f9,ffe82abeee50723e001a7d509f10e2ae,ffe8b8f8ebf7d08257925da4aef3a944,ffeb3a189aa893967ad3a1f30def6078,fffb75617eac6ae15aabc73eb7b6eb4e,fffc0c8ae125e5733e4f034e2a457b3c,fffc6f328cef53e76f5e5bb78ed23ba9,fffcf89ea054a29e92204955ca846d63,fffe1767dbc1713944851a0a4f02ec5b
8aec88cba08d53da65ab99cf20f6f9d9,0.461373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46baa1f69ac07779274bcd90b85d9a72,0.423688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45efa1f6bc65acc32bbbb953a1ed13b7,0.462260,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
e0ac9d926dda5e95162ef05adea7318c,0.363979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

# Evaluating Result 

## Our metric is MAP@K & Catalog Coverage
* getting the relevant jobs for each resume

In [31]:
hit_list = pd.read_csv('hits_list.csv', names=['resume', 'job_uniq_id'], index_col=0)
grouped_hit_list =  hit_list.groupby('resume').apply(lambda x : x.to_numpy().flatten().tolist()).to_dict()

In [32]:
# def precision_at_k(resume, k = 1):
#     recommended_jobs = generate_recommendations_for(resume,'temp')[0:k]
#     resume_hit_list = [1 if x in grouped_hit_list[resume] else 0 for x in recommended_jobs]
#     return resume_hit_list.count(1)/k

* needed functions for the calculation for our metrics

In [33]:
# note that the recommended jobs in this function is a product of cosine only, jaccard next time
def average_precision_at_k(resume, k = 1, algo='WE-UCF', matching_func = 'cosine'):
    recommendation_func = {
        'WE-UCF': generate_recommendations_for(resume,matching_func),
        'NN': nn_generate_recommendations_for(resume),
    }
    
    recommended_jobs = recommendation_func[algo][0:k]
    resume_hit_list = [1 if x in grouped_hit_list[resume] else 0 for x in recommended_jobs]
    
    total = 0
    rank = 1
    one_counter = 1
    
    for i in resume_hit_list:
        if i == 1:
            value = one_counter/rank
            total+=value
            one_counter += 1
        rank += 1
    
    try:
        return total / resume_hit_list.count(1)
        # return total / len(grouped_hit_list[resume])
    except ZeroDivisionError:
        return 0

In [34]:
def map_at_k(k=1, algo='WE-UCF', matching_func = 'cosine'):
    total = 0
    for resume_index in resume_no_dup.index:
        total += average_precision_at_k(resume_index, k = k, algo=algo, matching_func = matching_func)
    return total / len(resume_no_dup)

* sample MAP@K calculation , where k =  1

In [35]:
map_at_k(k = 1, algo='WE-UCF', matching_func = 'cosine')

KeyboardInterrupt: 

In [ ]:
map_at_k(k = 1, algo='WE-UCF', matching_func = 'jaccard') 

In [ ]:
map_at_k(k = 1, algo='NN', matching_func = 'cosine') 

In [ ]:
def catalog_coverage(recommendations, all_recommendations):
    return len(set(recommendations))/len(all_recommendations)
    

* sample coverage calculation , where k =  1

In [ ]:
recommendation_list = []
for i in resume_no_dup.index:
    # note  k = 2 here
    recommendation_list.extend(generate_recommendations_for(i, 'cosine')[0:2])
catalog_coverage(recommendation_list, no_duplicates_job)

In [ ]:
recommendation_list = []
for i in resume_no_dup.index:
    # note  k = 2 here
    recommendation_list.extend(generate_recommendations_for(i, 'jaccard')[0:2])
catalog_coverage(recommendation_list, no_duplicates_job)